# 优化模型参数
## 加载数据集

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## 定义模型

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 定义超参数
超参数是可调整的参数，可让您控制模型优化过程。不同的超参数值会影响模型训练和收敛速率
用于训练的超参数：
- Number of Epochs - 迭代数据集的次数
- Batch Size - 在更新参数之前通过网络传播的数据样本数
- Learning Rate - 学习率，在每个批次/epoch 更新模型参数的量。较小的值会导致学习速度变慢，而较大的值可能会导致训练期间出现不可预知的行为。

In [8]:
# 1e-3 = 1 * 10 ^ -3 = 0.001
learning_rate = 1e-3
batch_size = 64
# 训练轮次
epochs = 10

In [3]:
print(1e-3)

0.001


## 损失函数
当看到一些训练数据时，我们未经训练的网络可能无法给出正确答案。 损失函数测量获得的结果与目标值的差异程度，这是我们希望在训练期间最小化的损失函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实的数据标签值进行比较。

常见的损失函数包括 nn.MSELoss（均方误差）用于回归任务，以及 nn.用于分类的 NLLLoss （Negative Log Likelihood）。 nn.CrossEntropyLoss 结合了 nn.LogSoftmax 和 nn.NLLLoss 的

我们将模型的输出 logits 传递给 nn.CrossEntropyLoss，它将对 logit 进行归一化并计算预测误差。

In [9]:
# 初始化损失函数，使用交叉熵
loss_fn = nn.CrossEntropyLoss()

## 优化
优化是调整模型参数以减少每个训练步骤中的模型误差的过程。 优化算法定义了此过程的执行方式（在此示例中，我们使用随机梯度下降）。所有优化逻辑都封装在 optimizer 对象中。在这里，我们使用 SGD 优化器;此外，还有许多不同的优化器 在 PyTorch 中可用，例如 ADAM 和 RMSProp，它们更适合不同类型的模型和数据。

我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**在训练循环中，优化分三个步骤进行：**

- 调用 optimizer.zero_grad() 重置模型参数的梯度。默认情况下，梯度累加；为了防止重复计数，我们在每次迭代时都将它们显式归零。
- 通过调用 loss.backward() 反向传播预测损失。PyTorch 会根据每个参数来存储损失的梯度。
- 一旦我们有了梯度，我们调用 optimizer.step() 来根据向后传递中收集的梯度来调整参数。

# 全面实施
定义循环训练优化的代码 train_loop，以及根据测试数据评估模型性能的代码 test_loop。

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    # 数据样本数量
    size = len(dataloader.dataset)
    # 将模型设置为训练模式 - 这对批归一化和随机失活层很重要
    # 在这种情况下并非必需，但为了遵循最佳实践而添加
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 计算预测和损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播计算梯度
        loss.backward()
        # 优化参数
        optimizer.step()
        # 梯度归零
        optimizer.zero_grad()

        # 训练过程中，打印训练结果
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 将模型设置为评估模式 - 这对批归一化和随机失活层很重要
    # 在这种情况下并非必需，但为遵循最佳实践而添加
    model.eval()
    # 所有的数据样本个数
    size = len(dataloader.dataset)
    # 批次数：样本个数/batch_size。因为数据集一般非常大，不可能一次性加载到内存中，所以分批每次加载一部分。
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # 在测试模式下使用 torch.no_grad() 评估模型，可确保不会计算梯度。
    # 这一操作还能减少对那些设置了 requires_grad=True 的张量进行不必要的梯度计算，从而降低内存占用。
    with torch.no_grad():
        for X, y in dataloader:
            # 预测
            pred = model(X)
            # 计算损失并累加
            test_loss += loss_fn(pred, y).item()
            # 当前预测正确的个数
            # pred.argmax(1)：从维度1中获取值最大的索引（0行，1列）
            # .type(torch.float)：将boolean转换为float，True：1.0，False：0.0
            # .sum()：求和
            # .item()：取出数值，.sum()之后返回的是Tensor类型
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # 计算每批次平均损失
    test_loss /= num_batches
    # 计算正确率
    correct /= size
    # 打印测试结果
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 开始训练

In [12]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297567  [   64/60000]
loss: 2.291999  [ 6464/60000]
loss: 2.275599  [12864/60000]
loss: 2.267644  [19264/60000]
loss: 2.247663  [25664/60000]
loss: 2.229667  [32064/60000]
loss: 2.232101  [38464/60000]
loss: 2.203835  [44864/60000]
loss: 2.191503  [51264/60000]
loss: 2.173747  [57664/60000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.162530 

Epoch 2
-------------------------------
loss: 2.168622  [   64/60000]
loss: 2.164781  [ 6464/60000]
loss: 2.113581  [12864/60000]
loss: 2.127485  [19264/60000]
loss: 2.070771  [25664/60000]
loss: 2.029838  [32064/60000]
loss: 2.057753  [38464/60000]
loss: 1.989105  [44864/60000]
loss: 1.979263  [51264/60000]
loss: 1.919270  [57664/60000]
Test Error: 
 Accuracy: 58.1%, Avg loss: 1.915396 

Epoch 3
-------------------------------
loss: 1.945058  [   64/60000]
loss: 1.919714  [ 6464/60000]
loss: 1.811638  [12864/60000]
loss: 1.842107  [19264/60000]
loss: 1.720116  [25664/60000]
loss: 1.693116  [32064/600